In [19]:
import sys
sys.path.append('/minimax-dialogue/users/xiancai/verl')
import pickle

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/minimax-dialogue/users/xiancai/hf_models/R2EGym-32B-Agent')

def load_from_disk(filepath) -> 'DataProto':
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
        return data

batch = load_from_disk('/minimax-dialogue/users/xiancai/verl/sample_log/j-yhdv3fepym/batch_0001.pkl')

In [2]:
dir(batch)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'batch',
 'check_consistency',
 'chunk',
 'concat',
 'from_dict',
 'from_single_dict',
 'load_from_disk',
 'make_iterator',
 'meta_info',
 'non_tensor_batch',
 'pop',
 'print_size',
 'rename',
 'reorder',
 'repeat',
 'save_to_disk',
 'select',
 'select_idxs',
 'slice',
 'to',
 'union']

In [9]:
print(batch.batch.keys())

_StringKeys(dict_keys(['prompts', 'responses', 'response_mask', 'input_ids', 'attention_mask', 'position_ids', 'old_log_probs', 'entropys', 'ref_log_prob', 'token_level_scores', 'token_level_rewards', 'advantages', 'returns']))


In [20]:


entropy_list = batch.batch['entropys'][0]
token_list = batch.batch['responses'][0]
# token_list = [tokenizer.convert_ids_to_tokens(token) for token in token_list]

# assert len(entropy_list) == len(token_list)
# for entropy, token in zip(entropy_list, token_list):
#     print(entropy, token)

response_mask = batch.batch['response_mask'][0]
# compute average entropy of the masked tokens
# masked_entropy_list = [entropy for entropy, mask in zip(entropy_list, response_mask) if mask]
# print(sum(masked_entropy_list) / len(masked_entropy_list))

# not_masked_entropy_list = [entropy for entropy, mask in zip(entropy_list, response_mask) if not mask]
# print(sum(not_masked_entropy_list) / len(not_masked_entropy_list))
max_response_length = batch.batch['responses'].shape[-1]
loss_mask = batch.batch['attention_mask'][:, -max_response_length:][0]

print((entropy_list * loss_mask).sum() / (loss_mask.sum() + 1e-8))
print((entropy_list * response_mask).sum() / (response_mask.sum() + 1e-8))


tensor(0.6400)
tensor(0.1835)


In [ ]:
batch.batch['old_log_probs']

In [ ]:
batch.batch['token_level_rewards'].sum(-1).tolist()

In [26]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/minimax-dialogue/users/xiancai/hf_models/R2EGym-32B-Agent')


In [ ]:
data = batch.batch
responses = data['responses']

# print response_length each
max_response_length = batch.batch['responses'].shape[-1]

response_length = batch.batch['attention_mask'][:, -max_response_length:].sum(-1)
# response_length = responses.size(1)
# print the data if response_length > 32768
long_response_indices = (response_length > 32768).nonzero()

for idx in long_response_indices:
    response_ids = responses[idx].tolist()[0]
    print(response_ids)
    response = tokenizer.decode(response_ids)
    print(response)


# print(f"Number of samples with response_length > 32768: {len(long_response_indices)}")

# attention_mask = data['attention_mask']
# response_mask = data['response_mask']
# old_log_prob = data['old_log_probs']
# advantages = data['advantages']
# rewards = data['token_level_scores']

In [ ]:
long_response_indices

In [45]:
def highlight_response_with_ansi_colors(response_ids, response_mask, color_type="yellow"):
    """
    使用ANSI颜色代码为response添加彩色背景高亮
    
    Args:
        response_text: 解码后的response文本
        response_mask: 布尔mask，标记需要高亮的token位置
        color_type: 颜色类型 ("yellow", "red", "green", "blue", "cyan", "magenta")
    """
    # ANSI颜色代码映射
    color_codes = {
        "yellow": "\033[43m",     # 黄色背景
        "red": "\033[41m",        # 红色背景
        "green": "\033[42m",      # 绿色背景
        "blue": "\033[44m",       # 蓝色背景
        "cyan": "\033[46m",       # 青色背景
        "magenta": "\033[45m",    # 洋红色背景
        "white": "\033[47m",      # 白色背景
    }
    
    HIGHLIGHT = color_codes.get(color_type, color_codes["yellow"])
    RESET = "\033[0m"  # 重置颜色
    
    # 如果mask长度与文本长度不匹配，需要进行token级别的处理
    if len(response_mask) != len(response_ids):
        return highlight_tokens_with_ansi(response_ids, response_mask, HIGHLIGHT, RESET)
    
    result = []
    in_highlight = False
    
    tokens = tokenizer.convert_ids_to_tokens(response_ids)
    for i, char in enumerate(tokens):
        if response_mask[i] and not in_highlight:
            result.append(HIGHLIGHT)
            in_highlight = True
        elif not response_mask[i] and in_highlight:
            result.append(RESET)
            in_highlight = False
        
        result.append(char)
    
    if in_highlight:
        result.append(RESET)
    
    return ''.join(result)

def highlight_tokens_with_ansi(token_ids, token_mask, highlight_code, reset_code):
    """处理token级别的高亮"""
    # 这里需要根据您的tokenizer进行token分割
    # 假设使用空格分割（您可能需要根据实际情况调整）
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    print(tokens)

    
    if len(tokens) != len(token_mask):
        print(f"警告: tokens长度({len(tokens)})与mask长度({len(token_mask)})不匹配")
        return "".join(tokens)
    
    highlighted_tokens = []
    for token, should_highlight in zip(tokens, token_mask):
        if should_highlight:
            highlighted_tokens.append(f"{highlight_code}{token}{reset_code}")
        else:
            highlighted_tokens.append(token)
    
    return " ".join(highlighted_tokens)

In [ ]:
for idx in long_response_indices:
    response_ids = responses[idx].tolist()[0]
    response_mask = data['response_mask'][idx].tolist()[0]
    print(len(response_ids))
    print(sum(response_mask))
    print(response_ids)
    print(response_mask)
    print(tokenizer.convert_ids_to_tokens(response_ids))

    # response = tokenizer.decode(response_ids)
    # ANSI高亮测试
    ansi_result = highlight_response_with_ansi_colors(response_ids, response_mask, "red")

    # 保存测试文件
    with open("test_ansi.ans", "w", encoding="utf-8") as f:
        f.write(ansi_result)

    print("测试文件已保存: test_ansi.ans")

In [52]:
import torch
sequence_score = batch.batch['token_level_scores'].sum(-1)
sequence_reward = batch.batch['token_level_rewards'].sum(-1)

advantages = batch.batch['advantages']
returns = batch.batch['returns']

max_response_length = batch.batch['responses'].shape[-1]

prompt_mask = batch.batch['attention_mask'][:, :-max_response_length].bool()
response_mask = batch.batch['attention_mask'][:, -max_response_length:].bool()

max_prompt_length = prompt_mask.size(-1)

# response_info = _compute_response_info(batch)
# prompt_length = response_info['prompt_length']
# response_length = response_info['response_length']

valid_adv = torch.masked_select(advantages, response_mask)
valid_returns = torch.masked_select(returns, response_mask)

In [ ]:
log_probs = data['log_probs']
print(log_probs)
print(old_log_prob)

In [ ]:
l = batch.batch['token_level_scores'].sum(-1)
l = batch.batch['token_level_rewards'].sum(-1)
print(l)

In [ ]:
batch.batch['token_level_rewards'][0].tolist()

In [ ]:
import torch
torch.load('/data/minimax-dialogue/users/yuelan/repos/verl/a.pkl')